In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from pathlib import Path
import os
import nltk as nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
#tokenization for wordcloud, ect..
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from string import punctuation
import re

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/tylergehbauer/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [2]:
warner_df = pd.read_csv("../data/warner-bro.csv")

In [3]:
sentiments_warner = []#creates empty list

#Going to use VADER’s SentimentIntensityAnalyzer() to see how many 'Neutral (neu)' , 'Positive (pos)', and 'Negatvie(neg)'
# words there are for each article. 
for articles in warner_df['critics_consensus']: #goes through each artilce in df and applies polarity
    try:
        text = articles #accessing each articles 'content' which contains the article itself in 'bitcoin_news'
        results = analyzer.polarity_scores(text) #this is how get compound, pos, neg, and nue polarity scores
        #during imports we set analyzer equal to SentimentIntensityAnalyzer() 
        #'results' uses this function to find polarity score for each article in bitcoin_news
        compound = results["compound"] #gets compound polarity score
        pos = results["pos"] #gets postive polarity score
        neu = results["neu"] #gets neutral polarity score
        neg = results["neg"] #gets negative polarity score

        sentiments_warner.append({
            "Consensus": text,
            "Compound": compound,
            "Positive": pos,
            "Negative": neg,
            "Neutral": neu,
        }) #appends a dictionary inside the list-(sentiments) so a dataframe can be created
        #this appends all polarity score we got above
    except AttributeError:
        pass
#rinse and repeat for each article    
warner = pd.DataFrame(sentiments_warner)#creates a dataframe using the list of dicts
warner = warner.join(warner_df['movie_title'])
warner = warner.join(warner_df['original_release_date'])
warner = warner.join(warner_df['streaming_release_date'])
warner = warner.drop(columns = ['Positive', 'Negative', 'Neutral'])

In [4]:
warner[["original_release_date", "streaming_release_date"]] = warner[["original_release_date", "streaming_release_date"]].apply(pd.to_datetime)


In [5]:
warner.head()

,text,Compound,Positive,Negative,Neutral,movie_title,original_release_date,streaming_release_date
0,With attention strictly paid to style instead ...,0.2846,0.097,0.000,0.903,"10,000 B.C.",2008-03-07,2013-06-22
1,"If you saw Nine Queens, it may feel redundant,...",0.6652,0.207,0.000,0.793,The Breaking Point,1950-10-06,2016-08-25
2,Errol Flynn thrills as the legendary title cha...,0.5859,0.186,0.000,0.814,Criminal,2004-09-24,2016-04-27
3,"Kate Beckinsale is as lovely as ever, and does...",0.5719,0.171,0.044,0.785,The Adventures of Robin Hood,1938-05-14,2014-01-23
4,Although Darfur Now is not always engaging as ...,-0.4471,0.122,0.233,0.646,The Narrow Margin,1952-05-04,2012-08-20


In [6]:
warner = warner.drop(columns = ['Positive', 'Negative', 'Neutral'])

In [8]:
warner.head()

,text,Compound,movie_title,original_release_date,streaming_release_date
0,With attention strictly paid to style instead ...,0.2846,"10,000 B.C.",2008-03-07,2013-06-22
1,"If you saw Nine Queens, it may feel redundant,...",0.6652,The Breaking Point,1950-10-06,2016-08-25
2,Errol Flynn thrills as the legendary title cha...,0.5859,Criminal,2004-09-24,2016-04-27
3,"Kate Beckinsale is as lovely as ever, and does...",0.5719,The Adventures of Robin Hood,1938-05-14,2014-01-23
4,Although Darfur Now is not always engaging as ...,-0.4471,The Narrow Margin,1952-05-04,2012-08-20
